In [1]:
import h5py
import numpy as np
import json
import ltn
import torch
from torch.utils.data import Dataset, DataLoader
from collections import defaultdict
from torchvision.models import resnet50, ResNet50_Weights

In [2]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
device

'cpu'

In [3]:
BATCH_SIZE = 128
LEARNING_RATE = 0.00001
EPOCHS = 32

In [4]:
def get_tensor(idx, embedding_size=151):
    t = [0] * embedding_size
    t[idx] = 1
    t = torch.tensor(t)
    return t.to(torch.float32)

In [5]:
class VGImageDataset(Dataset):
    def __init__(
        self, imdb_h5, sgg_h5, sgg_dict, transform=None, target_transform=None
    ):
        self.imdb = h5py.File(imdb_h5)
        self.sgg = h5py.File(sgg_h5)
        with open(sgg_dict) as f:
            self.dicts = json.load(f)
            self.idx_to_labels = self.dicts["idx_to_label"]
            self.label_to_idx = self.dicts["label_to_idx"]
            self.idx_to_predicates = self.dicts["idx_to_predicate"]
            self.predicates_to_idx = self.dicts["predicate_to_idx"]
        self.transform = transform
        self.target_transform = target_transform

        def return_set():
            return set()

        self.logic_rules = defaultdict(return_set)

        for i in range(len(self.sgg["relationships"])):
            sub, obj, rel = (
                self.sgg["relationships"][i][0],
                self.sgg["relationships"][i][1],
                self.sgg["predicates"][i],
            )
            self.logic_rules[
                (
                    self.idx_to_labels[str(self.sgg["labels"][sub][0])].upper(),
                    self.idx_to_predicates[str(rel[0])].upper().replace(" ", "_"),
                )
            ].add(self.idx_to_labels[str(self.sgg["labels"][obj][0])].upper())

        for k in self.logic_rules:
            self.logic_rules[k] = list(self.logic_rules[k])

        self.g = {label: get_tensor(i) for i, label in enumerate(self.label_to_idx)}

    def __len__(self):
        return len(self.imdb["images"])

    def __getitem__(self, idx):
        image = self.imdb["images"][idx]
        if self.transform:
            image = self.transform(torch.tensor(image))
        image.to(torch.device(device))

        start = self.sgg["img_to_first_box"][idx]
        end = self.sgg["img_to_last_box"][idx]
        object_names = []
        if start > 0:
            object_names = [self.sgg["labels"][i][0] for i in range(start, end + 1)]
        for _ in range(len(object_names), 150):
            object_names += [150]

        return image, object_names

    def get_relations_grounding(self):
        dic = {
            predicate.upper().replace(" ", "_"): ltn.Constant(get_tensor(idx))
            for idx, predicate in enumerate(self.predicates_to_idx)
        }
        return dic

    def get_logic_rules(self):
        return self.logic_rules

    def get_object_grounding(self):
        dic = {
            obj.upper(): ltn.Constant(get_tensor(idx))
            for idx, obj in enumerate(self.label_to_idx)
        }
        return dic

    def colate_fn(self, data):
        batch = [d for d in data if d != None]
        return torch.utils.data.dataloader.default_collate(batch)

In [82]:
# TODO: Implement object predicate.


class Saved_Model:
    def __init__(self, path) -> None:
        weights = ResNet50_Weights.DEFAULT
        resnet = resnet50(weights=weights)
        resnet.fc = torch.nn.Linear(resnet.fc.in_features, 40)
        # self.model = resnet.load_state_dict(torch.load(path, map_location=device))

    def get_prob(self, x, label):
        # prediction = self.model(x)
        
        return torch.tensor([[1.0 for _ in range(label.shape[0]//BATCH_SIZE)] for _ in range(128)]) # TODO: Get probability of a class


saved_model_path = ""
model = Saved_Model(path=saved_model_path)
object_predicate = ltn.Predicate(func=lambda a, b: model.get_prob(a, b))

In [7]:
model_path = "<saved_model_directory>"
sgg_path = "../data/mini_VG-SGG.h5"
sgg_dict_path = "../data/mini_VG-SGG-dicts.json"
imdb_path = "../data/mini_imdb_1024.h5"
weights = ResNet50_Weights.DEFAULT
train_data = VGImageDataset(
    imdb_path, sgg_path, sgg_dict_path, transform=weights.transforms()
)
train_dataloader = DataLoader(
    train_data, BATCH_SIZE, shuffle=True, collate_fn=train_data.colate_fn
)

In [8]:
# we define connectives, quantifiers, and SatAgg
And = ltn.Connective(ltn.fuzzy_ops.AndProd())
Not = ltn.Connective(ltn.fuzzy_ops.NotStandard())
Implies = ltn.Connective(ltn.fuzzy_ops.ImpliesReichenbach())
Exists = ltn.Quantifier(ltn.fuzzy_ops.AggregPMean(p=2), quantifier="e")
Forall = ltn.Quantifier(ltn.fuzzy_ops.AggregPMeanError(p=2), quantifier="f")
SatAgg = ltn.fuzzy_ops.SatAgg()
Or = ltn.Connective(ltn.fuzzy_ops.OrProbSum())

In [9]:
class MLP(torch.nn.Module):
    def __init__(self, layer_sizes=(302, 250, 200, 151)):
        super(MLP, self).__init__()
        self.elu = torch.nn.ELU()
        self.sigmoid = torch.nn.Sigmoid()
        self.linear_layers = torch.nn.ModuleList(
            [
                torch.nn.Linear(layer_sizes[i - 1], layer_sizes[i])
                for i in range(1, len(layer_sizes))
            ]
        )
        self.softmax = torch.nn.Softmax(1)

    def forward(self, l, *x):
        x = list(x)
        if len(x) == 1:
            x = x[0]
        else:
            x = torch.cat(x, dim=1)
        for layer in self.linear_layers[:-1]:
            x = self.elu(layer(x))
        logits = self.linear_layers[-1](x)
        probs = self.softmax(logits)
        out = torch.sum(probs * l, dim=1)
        return out


relation_predicate = ltn.Predicate(MLP())

In [10]:
relation_grounding = train_data.get_relations_grounding()
obj_grounding = train_data.get_object_grounding()

In [11]:
params = list(relation_predicate.parameters())
optimizer = torch.optim.Adam(params, lr=0.00001)

In [ ]:
for epoch in range(1):
    for i, data in enumerate(train_dataloader, 0):
        images = data[0]
        logic_rules = train_data.get_logic_rules()
        axioms = []
        for k in logic_rules:
            sub = obj_grounding[k[0]]
            rel = relation_grounding[k[1]]
            objs = logic_rules[k]
            y = [obj_grounding[idx] for idx in objs]
            y = ltn.Variable('y', torch.stack([idx.value for idx in y]))
            x = ltn.Variable('x', images)
            axioms += [
                Forall(
                    [x, y],
                    Implies(
                        And(
                            object_predicate(x, sub),
                            relation_predicate(rel, sub, y),
                        ),
                        object_predicate(x, y),
                    ),
                )
            ]
        sat_agg = SatAgg(*axioms)
        loss = 1.0 - sat_agg
        loss.backward()
        optimizer.step()
        if epoch % 20 == 0:
            print(" epoch %d | loss %.4f | Train Sat %.3f " % (epoch, loss, sat_agg))